citygmlをcsv, geojsonに変換する　

In [ ]:
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import load_citygml

citygmlを読み込む

In [ ]:
filelist = list(Path('13100_tokyo23-ku_2020_citygml_3_2_op/udx/bldg/').glob('*.gml'))

In [ ]:
print(len(filelist))
print(filelist[0])

In [ ]:
df = pd.DataFrame()
for filepath in tqdm(filelist):
    tmpdf = load_citygml.citygml_to_dataframe(filepath)
    df = pd.concat([df, tmpdf]).reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
df.columns

いくつかのカラムがコードのままなのでmappingする

In [ ]:
def load_dict(dictpath:str) -> dict:
    return pd.read_xml(dictpath, 
                xpath='///gml:Definition',
                namespaces={'gml': 'http://www.opengis.net/gml'}
                ).astype(str).set_index('name')['description'].to_dict()

In [ ]:
dict_prefecture = load_dict('13100_tokyo23-ku_2020_citygml_3_2_op/codelists/Common_prefecture.xml')
df['prefecture_name'] = df['prefecture'].map(dict_prefecture)

In [ ]:
dict_city = load_dict('13100_tokyo23-ku_2020_citygml_3_2_op/codelists/Common_localPublicAuthorities.xml')
df['city_name'] = df['city'].map(dict_city)

In [ ]:
dict_zone = load_dict('13100_tokyo23-ku_2020_citygml_3_2_op/codelists/Common_districtsAndZonesType.xml')
df['districtsAndZonesType_name'] = df['districtsAndZonesType'].map(dict_zone)

In [ ]:
df['surveyYear'].unique()

In [ ]:
df['LOD1の立ち上げに使用する建築物の高さ'].unique()

In [ ]:
import japanize_matplotlib

In [ ]:
df['city_name'].value_counts().plot.bar()

In [ ]:
cols = ['id', '建物ID', 'measuredHeight', 'lod0RoofEdge', 'address', 'prefecture_name', 'city_name',
       '大字・町コード', '町・丁目コード', '13_区市町村コード_大字・町コード_町・丁目コード', 'buildingRoofEdgeArea',
       'districtsAndZonesType_name', 'surveyYear', '土砂災害警戒区域', '防火及び準防火地域', '地区計画', 'name', '高度地区']

In [ ]:
df['建物ID'].is_unique

csvに書き出す

In [ ]:
city_list = df['city_name'].unique()

In [ ]:
for city in tqdm(city_list):
    tmpdf = df.query('city_name == @city')[cols].dropna(axis=1, how='all').reset_index(drop=True)
    tmpdf.to_csv(f'csv/{city}_plateau.csv', index=False)

geojsonにする

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np

In [ ]:
# ポリゴンに変換する
df['lod0RoofEdge'] = df['lod0RoofEdge'].apply(lambda row: row.split())

In [ ]:
df['lod0RoofEdge_2d'] = df['lod0RoofEdge'].apply(lambda row: np.reshape(row, (-1,3)).astype(float))

In [ ]:
df['lod0RoofEdge_2d'] = df['lod0RoofEdge_2d'].apply(Polygon)

In [ ]:
cols = ['id', '建物ID', 'measuredHeight', 'lod0RoofEdge_2d', 'address', 'prefecture_name', 'city_name',
       '大字・町コード', '町・丁目コード', '13_区市町村コード_大字・町コード_町・丁目コード', 'buildingRoofEdgeArea',
       'districtsAndZonesType_name', 'surveyYear', '土砂災害警戒区域', '防火及び準防火地域', '地区計画', 'name', '高度地区']

In [ ]:
gdf = gpd.GeoDataFrame(df[cols])

In [ ]:
gdf.set_geometry('lod0RoofEdge_2d', crs='EPSG:6697', inplace=True)

In [ ]:
for city in tqdm(city_list):
    tmpdf = gdf.query('city_name == @city').dropna(axis=1, how='all').reset_index(drop=True)
    tmpdf.to_file(f'json/{city}_plateau.geojson', driver='GeoJSON', index=False)